In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings('ignore')

# ==============================================
# LOAD DATA (Preprocessing Awal)
# ==============================================
print("\n" + "="*70)
print("LOAD DATA PEMBELIAN DAN STOK")
print("="*70)

# 1. PARSING DATA PEMBELIAN
data = []
kode, nama, unit = None, None, None
with open('dataset-apotek-pembelian.tsv', 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        line = line.strip()
        if not line or set(line) == {'-'}:
            continue
        if re.match(r'^[A-Z0-9]{5,}\s+', line):
            parts = re.split(r'\s{2,}', line)
            kode = parts[0].strip()
            nama = parts[1].strip() if len(parts) > 1 else None
            unit = parts[-1].strip() if len(parts) > 2 else None
            continue
        if re.match(r'^\d{2}-\d{2}-\d{2}', line):
            tanggal = line[0:8].strip()
            no_transaksi = line[9:35].strip()
            qty_masuk = line[36:47].strip()
            nilai_masuk = line[48:61].strip()
            qty_keluar = line[62:73].strip()
            nilai_keluar = line[74:].strip()
            data.append([kode, nama, unit, tanggal, no_transaksi, qty_masuk, nilai_masuk, qty_keluar, nilai_keluar])

df = pd.DataFrame(data, columns=[
    'Kode', 'Nama_Produk', 'Unit', 'Tanggal', 'No_Transaksi',
    'Qty_Masuk', 'Nilai_Masuk', 'Qty_Keluar', 'Nilai_Keluar'
])

def to_float(val):
    val = str(val).replace('.', '').replace(',', '.')
    try:
        return float(val)
    except:
        return 0.0

for c in ['Qty_Masuk', 'Nilai_Masuk', 'Qty_Keluar', 'Nilai_Keluar']:
    df[c] = df[c].apply(to_float)

df['Tanggal'] = pd.to_datetime(df['Tanggal'], format='%d-%m-%y', errors='coerce')
df = df.dropna(subset=['Tanggal'])

# Tambahkan fitur temporal SEDERHANA (bukan agregasi)
df['Bulan'] = df['Tanggal'].dt.month
df['Tahun'] = df['Tanggal'].dt.year
df['Hari'] = df['Tanggal'].dt.day
df['Hari_dalam_Minggu'] = df['Tanggal'].dt.dayofweek

print(f"✓ Data mentah pembelian: {len(df)} transaksi")

# 2. PARSING DATA STOK
df_stok = pd.read_fwf('dataset-apotek-stok.tsv', encoding='utf-8')
df_stok = df_stok.dropna(axis=1, how='all')
df_stok = df_stok.loc[:, ~df_stok.columns.str.contains('Unnamed', case=False)]
df_stok.columns = (
    df_stok.columns.str.strip()
    .str.upper()
    .str.replace('.', '', regex=False)
    .str.replace(' ', '_', regex=False)
)

stok_col = [col for col in df_stok.columns if 'QTY' in col and 'STOK' in col]
if not stok_col:
    raise KeyError(f"Kolom stok tidak ditemukan!")
stok_col = stok_col[0]

df_stok = df_stok[~df_stok[stok_col].astype(str).str.contains('-', regex=False, na=False)]
df_stok = df_stok[df_stok[stok_col].astype(str).str.strip() != '']
df_stok[stok_col] = (
    df_stok[stok_col]
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

df_stok = df_stok.rename(columns={
    'KODE': 'Kode',
    'NAMA_PRODUK': 'Nama_Produk',
    'LOKASI': 'Lokasi',
    stok_col: 'Stok_Aktual',
    'UNIT': 'Unit'
})

print(f"✓ Data stok dimuat: {len(df_stok)} produk")

# 3. AGREGASI MINIMAL (HANYA UNTUK MERGE)
print("\n" + "="*70)
print("AGREGASI MINIMAL PER PRODUK (untuk merge dengan stok)")
print("="*70)

pembelian_simple = df.sort_values(['Kode', 'Tanggal']).groupby('Kode').tail(1).reset_index(drop=True)

print(f"✓ Mengambil transaksi TERAKHIR per produk: {len(pembelian_simple)} produk")

# Merge dengan stok
df_merged = pembelian_simple.merge(df_stok[['Kode', 'Stok_Aktual', 'Lokasi']], on='Kode', how='inner')

print(f"✓ Data merged: {len(df_merged)} produk")


# ==============================================
# FASE 1: DATA PREPROCESSING (Jurnal Section IV.A)
# ==============================================
print("\n" + "="*70)
print("FASE 1: DATA PREPROCESSING")
print("="*70)

# 1.1 Data Formatting
print("\n[1.1] Data Formatting")

# Encode Lokasi
le_lokasi = LabelEncoder()
df_merged['Lokasi_Encoded'] = le_lokasi.fit_transform(df_merged['Lokasi'].astype(str))

# Buat target: Kategori stok (Fast/Medium/Slow moving)
df_merged['Target'] = pd.cut(
    df_merged['Stok_Aktual'], 
    bins=3, 
    labels=[0, 1, 2]  # 0=Low, 1=Medium, 2=High stock
)

# Pilih fitur LANGSUNG dari data (TIDAK ada agregasi)
feature_cols = [
    'Qty_Masuk', 'Nilai_Masuk', 'Qty_Keluar', 'Nilai_Keluar',
    'Bulan', 'Tahun', 'Hari', 'Hari_dalam_Minggu', 
    'Stok_Aktual', 'Lokasi_Encoded'
]

X = df_merged[feature_cols].copy()
y = df_merged['Target'].copy()

# Remove missing
valid_idx = ~y.isna()
X = X[valid_idx].reset_index(drop=True)
y = y[valid_idx].reset_index(drop=True)

print(f"✓ Data Formatting selesai")
print(f"✓ Jumlah fitur: {X.shape[1]}")
print(f"✓ Jumlah sampel: {X.shape[0]}")
print(f"✓ Distribusi target: {y.value_counts().to_dict()}")

# 1.2 Data Scaling (Standardization - Equation 22 jurnal)
print("\n[1.2] Data Scaling (Standardization)")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=feature_cols, index=X.index)

print(f"✓ Standardization: mean=0, std=1")

# 1.3 Data Randomization
print("\n[1.3] Data Randomization")

y_encoded = y.astype(int).values

np.random.seed(42)
shuffle_idx = np.random.permutation(len(X_scaled))
X_scaled = X_scaled.iloc[shuffle_idx].reset_index(drop=True)
y_encoded = y_encoded[shuffle_idx]

print(f"✓ Data shuffled untuk menghindari bias")


# ==============================================
# FASE 2: FEATURE REDUCTION (BFS-RST - Algorithm 3)
# ==============================================
print("\n" + "="*70)
print("FASE 2: FEATURE REDUCTION USING BFS-RST")
print("="*70)

class BFS_RST_FeatureReduction:
    """
    Best-First Search + Rough Set Theory
    Implementasi Algorithm 3 dari jurnal
    """
    
    def __init__(self, min_features=3):
        self.min_features = min_features
        self.selected_features = None
        self.core_features = None
    
    def _compute_core_attributes(self, X, y):
        """
        Step: Compute core attributes using RST (Equation 17)
        Core = {a ∈ C : RED(C) ≠ RED(C - {a})}
        Approximation: gunakan mutual information
        """
        print("  → Computing Core Attributes (RST)...")
        mi_scores = mutual_info_classif(X, y, random_state=42)
        threshold = np.percentile(mi_scores, 70)
        core = np.where(mi_scores >= threshold)[0]
        print(f"  → Core attributes: {len(core)} features")
        return core, mi_scores
    
    def _evaluation_function(self, X_subset, y, features):
        """
        Evaluation function f(N) = g(N) + h(N) (Equation 20)
        """
        if len(features) == 0:
            return float('inf')
        
        rf = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42)
        rf.fit(X_subset, y)
        acc = rf.score(X_subset, y)
        
        # f(N) = alpha*cost + beta*error
        cost = len(features) / X_subset.shape[1]
        error = 1 - acc
        return 0.3 * cost + 0.7 * error
    
    def fit(self, X, y):
        """
        Main BFS-RST Algorithm
        """
        print("\n  Executing BFS-RST...")
        
        # Initialize: Core attributes
        core, mi_scores = self._compute_core_attributes(X, y)
        self.core_features = core
        
        # Priority Queue
        import heapq
        pq = []
        
        X_core = X.iloc[:, core]
        eval_core = self._evaluation_function(X_core, y, core)
        heapq.heappush(pq, (eval_core, tuple(core)))
        
        visited = set()
        best_features = core
        best_score = eval_core
        
        max_iter = 30
        iteration = 0
        
        print(f"  → BFS iterations (max: {max_iter})...")
        
        # BFS Loop
        while pq and iteration < max_iter:
            curr_score, curr_feat = heapq.heappop(pq)
            curr_feat = list(curr_feat)
            
            feat_tuple = tuple(sorted(curr_feat))
            if feat_tuple in visited:
                continue
            visited.add(feat_tuple)
            
            iteration += 1
            
            # Stopping criteria
            if len(curr_feat) <= self.min_features:
                if curr_score < best_score:
                    best_score = curr_score
                    best_features = curr_feat
                break
            
            # Generate child nodes (FR: remove one feature)
            for f in curr_feat:
                child = [x for x in curr_feat if x != f]
                
                if len(child) < self.min_features:
                    continue
                
                child_tuple = tuple(sorted(child))
                if child_tuple in visited:
                    continue
                
                X_child = X.iloc[:, child]
                child_score = self._evaluation_function(X_child, y, child)
                
                # Priority adjustment if reduct
                avg_mi_child = np.mean(mi_scores[child])
                avg_mi_all = np.mean(mi_scores)
                if avg_mi_child >= 0.8 * avg_mi_all:
                    child_score *= 0.8  # High priority
                
                heapq.heappush(pq, (child_score, tuple(child)))
                
                if child_score < best_score:
                    best_score = child_score
                    best_features = child
        
        self.selected_features = best_features
        print(f"  ✓ BFS-RST completed: {iteration} iterations")
        print(f"  ✓ Features selected: {len(self.selected_features)}")
        
        return self

# Execute BFS-RST
bfs_rst = BFS_RST_FeatureReduction(min_features=3)
bfs_rst.fit(X_scaled, y_encoded)

X_reduced = X_scaled.iloc[:, bfs_rst.selected_features]
reduced_names = [feature_cols[i] for i in bfs_rst.selected_features]

print(f"\n✓ Feature Reduction Done!")
print(f"  Original: {X_scaled.shape[1]} features")
print(f"  Reduced: {X_reduced.shape[1]} features")
print(f"  Selected: {reduced_names}")


# ==============================================
# FASE 3: FEATURE SELECTION (DCRRF - Algorithm 4)
# ==============================================
print("\n" + "="*70)
print("FASE 3: FEATURE SELECTION USING DCRRF")
print("="*70)

class DCRRF:
    """
    Dynamic Correlated Regularized Random Forest
    Implementasi Algorithm 4 dari jurnal
    """
    
    def __init__(self, n_estimators=50, lambda_reg=0.01, random_state=42):
        self.n_estimators = n_estimators
        self.lambda_reg = lambda_reg
        self.random_state = random_state
        self.feature_sets = []
        self.optimal_features = None
        self.feature_freq = None
    
    def _cfs_merit(self, X, y, features):
        """
        CFS criterion (Equation 18, 25)
        Merit_S = k*rcf / sqrt(k + k(k-1)*rff)
        """
        if len(features) == 0:
            return 0
        
        X_sub = X.iloc[:, features]
        k = len(features)
        
        # rcf: correlation with class
        mi = mutual_info_classif(X_sub, y, random_state=self.random_state)
        rcf = np.mean(mi)
        
        # rff: inter-feature correlation
        if k > 1:
            corr = X_sub.corr().abs()
            mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
            rff = corr.where(mask).stack().mean()
        else:
            rff = 0
        
        denom = np.sqrt(k + k * (k - 1) * rff)
        merit = (k * rcf) / denom if denom > 0 else 0
        
        return merit
    
    def _select_features_cfs(self, X, y, max_features):
        """
        Greedy forward selection using CFS
        """
        selected = []
        remaining = list(range(X.shape[1]))
        
        for _ in range(min(max_features, len(remaining))):
            best_merit = -1
            best_feat = None
            
            for f in remaining[:5]:  # Limit search untuk efisiensi
                candidate = selected + [f]
                merit = self._cfs_merit(X, y, candidate)
                
                if merit > best_merit:
                    best_merit = merit
                    best_feat = f
            
            if best_feat is not None:
                selected.append(best_feat)
                remaining.remove(best_feat)
        
        return selected
    
    def fit(self, X, y):
        """
        Main DCRRF Algorithm
        """
        print("\n  Executing DCRRF...")
        np.random.seed(self.random_state)
        
        n_samples = X.shape[0]
        n_features = X.shape[1]
        
        self.feature_freq = np.zeros(n_features)
        
        print(f"  → Training {self.n_estimators} trees...")
        
        for t in range(self.n_estimators):
            # Bootstrap Sample
            boot_idx = np.random.choice(n_samples, size=n_samples, replace=True)
            X_boot = X.iloc[boot_idx]
            y_boot = y[boot_idx]
            
            # Dynamic FS dengan CFS
            selected = self._select_features_cfs(
                X_boot, y_boot, 
                max_features=max(2, n_features // 2)
            )
            
            self.feature_sets.append(selected)
            
            for f in selected:
                self.feature_freq[f] += 1
            
            if (t + 1) % 10 == 0:
                print(f"    → {t+1}/{self.n_estimators} trees trained")
        
        # Determine Optimal Features (Equation 27: Intersection)
        # Pilih fitur yang muncul di >= 50% trees
        threshold = self.n_estimators * 0.5
        self.optimal_features = np.where(self.feature_freq >= threshold)[0]
        
        if len(self.optimal_features) < 2:
            # Fallback: ambil top features
            self.optimal_features = np.argsort(self.feature_freq)[-3:]
        
        print(f"\n  ✓ DCRRF completed!")
        print(f"  ✓ Optimal features: {len(self.optimal_features)}")
        
        return self

# Execute DCRRF
dcrrf = DCRRF(n_estimators=50, lambda_reg=0.01, random_state=42)
dcrrf.fit(X_reduced, y_encoded)

X_final = X_reduced.iloc[:, dcrrf.optimal_features]
final_names = [reduced_names[i] for i in dcrrf.optimal_features]

print(f"\n✓ Feature Selection Done!")
print(f"  After BFS-RST: {X_reduced.shape[1]} features")
print(f"  After DCRRF: {X_final.shape[1]} features")
print(f"  Final features: {final_names}")


# ==============================================
# FASE 4: DATA ANALYSIS (SVM - Section IV.D)
# ==============================================
print("\n" + "="*70)
print("FASE 4: DATA ANALYSIS USING SVM")
print("="*70)

# Split 80:20 (Table II jurnal)
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\n[4.1] Train-Test Split")
print(f"  Train: {len(X_train)} samples")
print(f"  Test: {len(X_test)} samples")

# SVM dengan hyperparameter Table II
print(f"\n[4.2] SVM Hyperparameters (Table II):")
print(f"  - Kernel: RBF")
print(f"  - C: 1")
print(f"  - Max iterations: 100")

svm_model = SVC(kernel='rbf', C=1, max_iter=100, random_state=42)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

# Metrics (Table III)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
sensitivity = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

# Specificity (manual)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
specificity_list = []
for i in range(len(cm)):
    tn = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
    fp = np.sum(cm[:, i]) - cm[i, i]
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_list.append(spec)
specificity = np.mean(specificity_list)


# ==============================================
# HASIL AKHIR (Format Table III Jurnal)
# ==============================================
print("\n" + "="*70)
print("HASIL AKHIR - PERFORMANCE COMPARISON (Table III Format)")
print("="*70)

print(f"\n{'Model':<20} {'FS':<5} {'Accuracy':<10} {'Sensitivity':<12} {'Specificity':<12} {'Precision':<10} {'F1-score':<10}")
print("="*90)
print(f"{'BFSRST+DCRRF':<20} {len(final_names):<5} {accuracy:.4f}     {sensitivity:.4f}       {specificity:.4f}       {precision:.4f}    {f1:.4f}")

print(f"\n✓ Feature Selection Summary:")
print(f"  - Original features: {len(feature_cols)}")
print(f"  - After BFS-RST: {len(reduced_names)}")
print(f"  - After DCRRF: {len(final_names)}")
print(f"  - Reduction: {len(feature_cols) - len(final_names)} features ({((len(feature_cols)-len(final_names))/len(feature_cols)*100):.1f}%)")

print(f"\n✓ Final Selected Features:")
for i, feat in enumerate(final_names, 1):
    freq = dcrrf.feature_freq[dcrrf.optimal_features[i-1]]
    pct = (freq / dcrrf.n_estimators) * 100
    print(f"  {i}. {feat:<25} (selected in {pct:.1f}% of trees)")

# Baseline comparison
print("\n" + "="*70)
print("COMPARISON WITH BASELINE (All Features)")
print("="*70)

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

svm_baseline = SVC(kernel='rbf', C=1, max_iter=100, random_state=42)
svm_baseline.fit(X_train_all, y_train_all)
y_pred_base = svm_baseline.predict(X_test_all)

acc_base = accuracy_score(y_test_all, y_pred_base)
prec_base = precision_score(y_test_all, y_pred_base, average='weighted', zero_division=0)
sens_base = recall_score(y_test_all, y_pred_base, average='weighted', zero_division=0)
f1_base = f1_score(y_test_all, y_pred_base, average='weighted', zero_division=0)

print(f"\n{'Model':<20} {'Features':<10} {'Accuracy':<12} {'Precision':<12} {'Sensitivity':<12} {'F1-Score':<10}")
print("="*80)
print(f"{'Baseline (All)':<20} {len(feature_cols):<10} {acc_base:.4f}       {prec_base:.4f}       {sens_base:.4f}       {f1_base:.4f}")
print(f"{'BFSRST+DCRRF':<20} {len(final_names):<10} {accuracy:.4f}       {precision:.4f}       {sensitivity:.4f}       {f1:.4f}")

print(f"\n✓ Improvement:")
print(f"  - Feature reduction: {((len(feature_cols)-len(final_names))/len(feature_cols)*100):.1f}%")
print(f"  - Accuracy change: {(accuracy-acc_base):+.4f} ({((accuracy-acc_base)/acc_base*100):+.2f}%)")
print(f"  - F1-Score change: {(f1-f1_base):+.4f} ({((f1-f1_base)/f1_base*100):+.2f}%)")

print("\n" + "="*70)
print("✓ IMPLEMENTASI JURNAL SELESAI!")
print("="*70)


LOAD DATA PEMBELIAN DAN STOK
✓ Data mentah pembelian: 138364 transaksi
✓ Data stok dimuat: 1518 produk

AGREGASI MINIMAL PER PRODUK (untuk merge dengan stok)
✓ Mengambil transaksi TERAKHIR per produk: 2024 produk
✓ Data merged: 359 produk

FASE 1: DATA PREPROCESSING

[1.1] Data Formatting
✓ Data Formatting selesai
✓ Jumlah fitur: 10
✓ Jumlah sampel: 359
✓ Distribusi target: {0: 350, 1: 6, 2: 3}

[1.2] Data Scaling (Standardization)
✓ Standardization: mean=0, std=1

[1.3] Data Randomization
✓ Data shuffled untuk menghindari bias

FASE 2: FEATURE REDUCTION USING BFS-RST

  Executing BFS-RST...
  → Computing Core Attributes (RST)...
  → Core attributes: 3 features
  → BFS iterations (max: 30)...
  ✓ BFS-RST completed: 1 iterations
  ✓ Features selected: 3

✓ Feature Reduction Done!
  Original: 10 features
  Reduced: 3 features
  Selected: ['Qty_Masuk', 'Nilai_Masuk', 'Stok_Aktual']

FASE 3: FEATURE SELECTION USING DCRRF

  Executing DCRRF...
  → Training 50 trees...
    → 10/50 trees tra